# Node classification for Heterogeneous graph

## 0. Enviroment setup

In [1]:
# !pip uninstall torch torchvision torchaudio --yes
# !pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121
# !pip install lightning torch_geometric
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
# !pip install wandb

In [2]:
import os
import shutil
import wandb
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
# from torch_geometric.loader import LinkNeighborLoader
# from torch_geometric.data.lightning import LightningLinkData

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger

In [3]:
from network_analysis.models.node_classification import *
from network_analysis.dataloaders import *

In [4]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Data Pre-processing

In [6]:
DATA_VERSION = 'final_v1'

In [7]:
# data = getHiveDataset(
#     f'dataset/hive/{DATA_VERSION}/nodes_labelled.csv',
#     f'dataset/hive/{DATA_VERSION}/edges_labelled.csv',
# )
# data = T.ToUndirected(merge=False)(data)
data = torch.load(f'dataset/hive/{DATA_VERSION}/hive.pt')
data

HeteroData(
  user={
    x=[18645, 5],
    y=[18645],
  },
  comment={
    x=[125111, 1],
    y=[125111],
  },
  post={
    x=[13540, 1],
    y=[13540],
  },
  (user, upvote, comment)={
    edge_index=[2, 423638],
    edge_attr=[423638, 1],
    y=[423638],
  },
  (user, upvote, post)={
    edge_index=[2, 554131],
    edge_attr=[554131, 1],
    y=[554131],
  },
  (user, write, comment)={
    edge_index=[2, 78696],
    edge_attr=[78696, 1],
    y=[78696],
  },
  (user, write, post)={
    edge_index=[2, 12958],
    edge_attr=[12958, 1],
    y=[12958],
  },
  (user, downvote, comment)={
    edge_index=[2, 6819],
    edge_attr=[6819, 1],
    y=[6819],
  },
  (user, downvote, post)={
    edge_index=[2, 2934],
    edge_attr=[2934, 1],
    y=[2934],
  },
  (comment, belong_to, comment)={
    edge_index=[2, 58911],
    edge_attr=[58911, 1],
    y=[58911],
  },
  (comment, belong_to, post)={
    edge_index=[2, 19838],
    edge_attr=[19838, 1],
    y=[19838],
  },
  (comment, rev_upvote, user)={


In [8]:
num_edges = len(data.edge_types)
edge_types, rev_edge_types = data.edge_types[:num_edges//2], data.edge_types[num_edges//2:]

In [9]:
edge_types

[('user', 'upvote', 'comment'),
 ('user', 'upvote', 'post'),
 ('user', 'write', 'comment'),
 ('user', 'write', 'post'),
 ('user', 'downvote', 'comment'),
 ('user', 'downvote', 'post'),
 ('comment', 'belong_to', 'comment'),
 ('comment', 'belong_to', 'post')]

In [10]:
transform = T.RandomNodeSplit(
    num_val=0.1,
    num_test=0.1,
)

data = transform(data)

## 2. Training

In [15]:
models = {
    # 'HGT': HGT, 
    # 'GATv2': GATv2, 
    # 'GraphSAGE': GraphSAGE, 
    'GAT':GAT
}

In [16]:
models = { k: m(
    in_channels=-1,  
    hidden_channels=[64, 128, 128, 256, 256], 
    metadata=data.metadata(), 
    edge_types=edge_types,
    rev_edge_types=rev_edge_types,
    # aggr_scheme='mean',
) for k, m in models.items()}

In [17]:
class GraphDataset(Dataset):
    def __init__(
        self,
        data,
        node_types,
        mask_key,
    ):
        self.data = data
        self.node_types = node_types
        self.train_mask = [data[t][mask_key] for t in node_types]

    def __len__(self):
        return len(self.node_types)

    def __getitem__(self, idx):
        return self.data, self.node_types[idx], self.train_mask[idx]
    
def collate_fn(input):
    data, node_types, mask = zip(*input)
    return data[0], node_types, mask

train_loader = DataLoader(
    GraphDataset(data, data.metadata()[0], 'train_mask'),
    batch_size=1,
    shuffle=True,
    drop_last=False,
    pin_memory=False,
    num_workers=4,
    collate_fn=collate_fn,
)
val_loader = DataLoader(
    GraphDataset(data, data.metadata()[0], 'val_mask'),
    batch_size=3,
    shuffle=False,
    drop_last=False,
    pin_memory=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [18]:
for mtype, model in models.items():
    log_dir = 'results/log/nc/' + mtype.lower()
    loss_checkpoint_dir = f'results/checkpoints/nc/{mtype.lower()}/loss'
    auc_checkpoint_dir = f'results/checkpoints/nc/{mtype.lower()}/roc_auc'
    acc_checkpoint_dir = f'results/checkpoints/nc/{mtype.lower()}/acc'

    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(loss_checkpoint_dir, exist_ok=True)
    os.makedirs(auc_checkpoint_dir, exist_ok=True)
    os.makedirs(acc_checkpoint_dir, exist_ok=True)
    

    lr = 1e-3
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    model.set_optimizer(optim)

    wandb_logger = WandbLogger(
        project="NodeClassification_finalv1",
        log_model=True,
        save_dir=log_dir,
        name=mtype,
        entity='ssc_project'

    )

    loss_checkpoint_callback = ModelCheckpoint(
        monitor=f'val_loss',
        dirpath=loss_checkpoint_dir,
        filename='NodeClsf-{epoch:02d}-{val_loss:.2f}',
        save_top_k=3,
        save_last=True,
        mode='min',
        every_n_epochs=1
    )
    roc_auc_checkpoint_callback = ModelCheckpoint(
        monitor=f'val_roc_auc',
        dirpath=auc_checkpoint_dir,
        filename='NodeClsf-{epoch:02d}-{val_roc_auc:.2f}',
        save_top_k=3,
        save_last=True,
        mode='max',
        every_n_epochs=1
    )
    acc_checkpoint_callback = ModelCheckpoint(
        monitor=f'val_accuracy',
        dirpath=acc_checkpoint_dir,
        filename='NodeClsf-{epoch:02d}-{val_accuracy:.2f}',
        save_top_k=3,
        save_last=True,
        mode='max',
        every_n_epochs=1
    )

    trainer = L.Trainer(
        max_epochs=500,
        check_val_every_n_epoch=10,
        callbacks=[
            loss_checkpoint_callback, 
            roc_auc_checkpoint_callback,
            acc_checkpoint_callback,
        ],
        logger=wandb_logger,
        log_every_n_steps=1
    )

    trainer.fit(model, train_loader, val_loader)
    wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params
----------------------------------------------
0 | encoder | GraphModule       | 2.0 M 
1 | crit    | BCEWithLogitsLoss | 0     
2 | mlp     | ModuleDict        | 493 K 
----------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.998     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (125111x1 and 5x64)